<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - KNN User User MRR</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, user_knn as knn
from lenskit import topn
import pandas as pd
import numpy as np

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [4]:
def model_trainer(neighbours,center):
    neighbours=int(neighbours)
    all_recs = []
    test_data = []
    algo_uu = knn.UserUser(neighbours,center=center,aggregate='weighted-average')
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test)
        all_recs.append(eval('UserUser', algo_uu, train, test))
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.recip_rank)
    results = rla.compute(all_recs, test_data)
    return results["recip_rank"].mean()

In [5]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    neighbours = params['neighbours']
    center = params['center']
    metric = model_trainer(neighbours,center)
    print([neighbours,center,metric],',')
    return {'loss': -metric, 'status': STATUS_OK }

space= {'neighbours': hp.uniform('neighbours', 1, 100),
       'center': hp.choice('center', ['True', 'False'])}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

[83.49946312816014, 'True', 0.024198272655178533]                                                                      
,                                                                                                                      
[64.79650778481957, 'True', 0.024164259955920118]                                                                      
,                                                                                                                      
[88.17703503116006, 'True', 0.02458699540370699]                                                                       
,                                                                                                                      
[77.19550264602454, 'True', 0.025020090869953274]                                                                      
,                                                                                                                      
[93.98415869208051, 'False', 0.021542755

[63.851425555728724, 'False', 0.024422054437276106]                                                                    
,                                                                                                                      
[10.715199339712562, 'False', 0.037102650136701686]                                                                    
,                                                                                                                      
[6.040434031271012, 'False', 0.04620154456803677]                                                                      
,                                                                                                                      
[18.348619517266624, 'False', 0.03328616943826039]                                                                     
,                                                                                                                      
[56.19755418836726, 'False', 0.024504630

[3.67539348996244, 'True', 0.06495629757390643]                                                                        
,                                                                                                                      
[19.49418415091714, 'True', 0.03153973260863474]                                                                       
,                                                                                                                      
[7.9405147697996705, 'True', 0.04888440110149301]                                                                      
,                                                                                                                      
[1.633617126065277, 'True', 0.06505300612191045]                                                                       
,                                                                                                                      
[23.84759827042505, 'True', 0.0317011899

In [7]:
metric = pd.DataFrame(array)
metric.columns = ['neighbours','center','MRR']
metric

,neighbours,center,MRR
0,83.499463,True,0.024198
1,64.796508,True,0.024164
2,88.177035,True,0.024587
3,77.195503,True,0.025020
4,93.984159,False,0.021543
...,...,...,...
95,78.074195,True,0.026228
96,13.583387,True,0.033984
97,4.095396,False,0.056735
98,29.101255,True,0.030094


In [8]:
metric.sort_values(by=['MRR'],ascending=False).head(10)

,neighbours,center,MRR
65,2.896923,True,0.076597
29,2.140672,False,0.076575
51,1.481681,True,0.075266
61,1.095879,True,0.072737
88,1.552442,False,0.072689
39,1.118201,False,0.072396
21,3.057396,False,0.068806
20,1.153792,False,0.067747
11,3.167951,False,0.067011
75,3.994068,True,0.066118


In [9]:
metric.to_csv('metric_MRR.csv', index=False)